In [96]:
import pandas as pd
from scipy.optimize import minimize
import plotly.graph_objects as go
import numpy as np
from math import e
ln = np.log  
ln(e)

1.0

In [97]:
df = pd.read_excel("extrem.xlsx")

In [98]:
df.head()

,Datetime,Unnamed: 1,temp,Unnamed: 3,tnz,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,tt,t_tenzo,tnzl,dif
0,2024-03-18 14:06:40,8,34.6248,26.800,2183140,2178584,2178557,0,0,42.61,36.4642,26,52.15,0,-163,35.108667,2183140,0
1,2024-03-18 14:06:52,8,35.1197,24.120,2183170,2178549,2178525,0,0,38.31,36.9437,24,52.47,0,-861,35.574000,2183170,30
2,2024-03-18 14:08:05,8,35.5940,21.708,2183159,2178476,2178503,0,0,37.17,37.4262,-27,52.73,0,-1603,36.068667,2183159,-11
3,2024-03-18 14:09:17,8,36.0510,19.537,2183152,2178408,2178484,0,0,37.02,37.8699,-75,52.96,0,-2266,36.510667,2183152,-7
4,2024-03-18 14:10:30,8,36.4945,17.584,2183170,2178368,2178466,0,0,37.15,38.3060,-98,53.17,0,-2935,36.956667,2183170,18


In [99]:
desired_columns = ['Datetime','t_tenzo','tnzl']

In [100]:
df = df.drop(df.columns.difference(desired_columns),axis=1)

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909 entries, 0 to 908
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Datetime  909 non-null    datetime64[ns]
 1   t_tenzo   909 non-null    float64       
 2   tnzl      909 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 21.4 KB


In [102]:
df.head()

,Datetime,t_tenzo,tnzl
0,2024-03-18 14:06:40,35.108667,2183140
1,2024-03-18 14:06:52,35.574000,2183170
2,2024-03-18 14:08:05,36.068667,2183159
3,2024-03-18 14:09:17,36.510667,2183152
4,2024-03-18 14:10:30,36.956667,2183170


In [103]:
def create_plot(df):
    df.loc[150:, 'tnzl'] += 80
    df['t_tenzo'] = df['t_tenzo'].shift(periods=20)
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df['Datetime'], y=df['tnzl'], mode='lines', name='tenzo'))
    fig.add_trace(go.Scatter(x=df['Datetime'], y=df['t_tenzo'], mode='lines', name='Temp', yaxis='y2'))

    fig.update_layout(
        title='График Temp и tenzo от времени',
        title_x=0.5,
        xaxis=dict(title='Datetime'),
        yaxis=dict(tickformat='d', title='Tenzo', side='left', visible=True),
        yaxis2=dict(tickformat='d', title='Temp', overlaying='y', side='right', visible=True),
        hovermode="x",
        font=dict(size=14),
        margin=dict(l=100, r=100, t=50, b=50)
    )

    fig.show()


In [104]:
create_plot(df)

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909 entries, 0 to 908
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Datetime  909 non-null    datetime64[ns]
 1   t_tenzo   889 non-null    float64       
 2   tnzl      909 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 21.4 KB


In [106]:
pearson_corr = df['t_tenzo'].corr(df['tnzl'], method='pearson')
spearman_corr = df['t_tenzo'].corr(df['tnzl'], method='spearman')

print("Коэффициент Пирсона:", pearson_corr)
print("Коэффициент Спирмена:", spearman_corr)

Коэффициент Пирсона: 0.9145126938882244
Коэффициент Спирмена: 0.9260871862826145


In [107]:
def objective_function(C, df):

    t_exp = df.t_tenzo * (1 - np.exp(-abs(C[0])))  

    U_t_exp = df.tnzl - C[1] * t_exp ** 2

    dt = df['t_tenzo'].diff() 

    U_dt = U_t_exp  - C[3] * dt

    U_ln = U_dt - C[4] * np.log(t_exp)  

    dU = U_ln.diff()

    U_dU = U_ln - dU ** 2 * C[5]

    dU_dU = U_dU.diff()
    d2u_dU = dU_dU.diff()

    U_d2U_dU = U_dU - d2u_dU ** 2 * C[6]

    dt_exp = t_exp.diff()
    d2t_exp = dt_exp.diff()

    U_d2t_exp = U_d2U_dU - d2t_exp * C[7] + d2t_exp ** 2 * C[8]

    var_level = np.var(U_d2t_exp)
    
    return var_level


initial_guess = [0.0364, 0.0002, 50, 27, 1913, 0.0039, 0.00011, 6.67, 0.56]

result = minimize(objective_function, initial_guess, args=(df,), method='BFGS')

optimal_K = result.x
optimal_scalar_value = objective_function(optimal_K, df)

print('Оптимальные значения K:', pd.DataFrame(optimal_K))
print('Оптимальные значения дисперсии:', optimal_scalar_value)


Оптимальные значения K:              0
0     0.556021
1    -0.490523
2    50.000000
3 -1591.397723
4  2086.154887
5     0.000950
6    -0.000399
7  1898.842444
8    -3.495417
Оптимальные значения дисперсии: 1283.986140482924


In [108]:
def function(C, df):

    t_exp = df.t_tenzo * (1 - np.exp(-abs(C[0])))  

    U_t_exp = df.tnzl - C[1] * t_exp ** 2

    dt = df['t_tenzo'].diff() 

    U_dt = U_t_exp - C[3] * dt

    U_ln = U_dt - C[4] * np.log(t_exp)  

    dU = U_ln.diff()

    U_dU = U_ln - dU ** 2 * C[5]

    dU_dU = U_dU.diff()
    d2u_dU = dU_dU.diff()

    U_d2U_dU = U_dU - d2u_dU ** 2 * C[6]

    dt_exp = t_exp.diff()
    d2t_exp = dt_exp.diff()

    U_d2t_exp = U_d2U_dU - d2t_exp * C[7] + d2t_exp ** 2 * C[8]

    return U_d2t_exp

level_dHdt = function(result.x,df)
# level_dHdt_without_nan = level_dHdt.dropna()  

optimal_K_list = [optimal_K]
optimal_K_df = pd.DataFrame(optimal_K_list,columns=['K{}'.format(i) for i in range(0, 9)])
optimal_K_df.to_csv('K0.csv', mode='a', header=False, index=False)


In [109]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['Datetime'], y=df.tnzl, mode='lines', name='Tenzo Data'))
fig.add_trace(go.Scatter(x=df['Datetime'], y=level_dHdt, mode='lines', name='Level Data', yaxis='y2'))
    
  
fig.update_layout(
    legend=dict(x=.8),
    title='График сравнения  Tenzo и Level',
    title_x=0.5,
    xaxis=dict(title='Datatime'),
    yaxis=dict(tickformat ='d',title='Tenzo Data', side='left'),
    yaxis2=dict(tickformat ='d',title='Level Data', overlaying='y', side='right'),
    hovermode="x",
    font=dict(size=14),
    margin=dict(l=100, r=100, t=50, b=50) 
    )
fig.show()


In [110]:
delta_level = np.nanmax(level_dHdt) - np.nanmin(level_dHdt)
print("Разница уровней =", delta_level)


Разница уровней = 336.75213307281956


In [111]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['Datetime'], y=df.tnzl, mode='lines', name='Tenzo Data'))
fig.add_trace(go.Scatter(x=df['Datetime'], y=level_dHdt, mode='lines', name='Level Data'))
    
  
fig.update_layout(
    legend=dict(x=.8),
    title='График сравнения  Tenzo и Level',
    title_x=0.5,
    xaxis=dict(title='Datatime'),
    yaxis=dict(tickformat ='d',title='Tenzo Data', side='left'),
    hovermode="x",
    font=dict(size=14),
    margin=dict(l=100, r=100, t=50, b=50) 
    )
fig.show()
